# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
#Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [14]:
##Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = 'select artist, song, length from music_library where sessionId = 338 and itemInSession = 4';
                    

In [21]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "CREATE TABLE IF NOT EXISTS music_library"
        query = query + "(sessionId int, itemInSession int, artist text, song text, length float, userID int, PRIMARY KEY(sessionId, itemInSession))"
        session.execute(query)
        ## Assign which column element should be assigned for each column in the INSERT statement.
        query = "INSERT INTO music_library (sessionId, itemInSession, artist, song, length, userID)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]), int(line[10])))

#### Do a SELECT to verify that the data have been inserted into each table

In [23]:
## Add in the SELECT statement to verify the data was entered into the table
query = 'select artist, song, length from music_library where sessionId = 338 and itemInSession = 4';


rows = session.execute(query)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


Here the Primary Key has two fields: sessionId is the partition key, and itemInSession is clustering key. Partitioning is done by sessionId and within that partition, rows are ordered by the itemInSession. The artist, song, and length where sessionID = 338 and itemInSession = 4 is Faithless (artist) - Music Matters (Mark Knight dub) (song). This song ran for a length of 495.31 seconds.

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [17]:
##Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "CREATE TABLE IF NOT EXISTS songs"
        query = query + "(userId int,sessionId int,itemInSession int, artist text, song text, firstName text,lastName text,PRIMARY KEY ((userId, sessionId), itemInSession))"
        session.execute(query)

        query = "INSERT INTO songs (userId, firstName, sessionId, itemInSession, artist, song, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), line[1], int(line[8]), int(line[3]), line[0], line[9], line[4]))
    
query = 'select artist, song, firstName, lastName from songs where userId=10 and sessionId=182 order by itemInSession';


rows = session.execute(query)

for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)                 

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


Here the Primary Key has three fields: userId and sessionId are the partition key, and itemInSession is clustering key. Partitioning is done by userId and sessionId and within that partition, rows are ordered by the itemInSession. In session 182 with user 10, Sylyive Cruz listened to 4 songs: Down To The Bone's Keep On Keepin' On, Three Drives's Greece 2000, Sebastien Tellier's  Kilometer and Lonnie Gordon's Catch You Baby (Steve Pitron & Max Sanna Radio Edit).

In [24]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "CREATE TABLE IF NOT EXISTS allHands"
        query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
        session.execute(query)

        query = "INSERT INTO allHands (song,  userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (line[9], int(line[10]),line[1], line[4]))
query = "select firstName, lastName from allHands where song='All Hands Against His Own'";
rows = session.execute(query)

for row in rows:
    print (row.firstname, row.lastname)   
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


Here the Primary Key has three fields: songId is the partition key, and userId is clustering key. Partitioning is done by song and within that partition, userId are ordered by the itemInSession.Users Jacqueline Lynch, Sara Johnson, and Tegan Levine all listened to 'All Hands Against His Own'

### Drop the tables before closing out the sessions

In [4]:
## Drop the table before closing out the sessions

In [16]:
query = "DROP table music_library"
rows = session.execute(query)

In [100]:
query = "DROP table songs"
rows = session.execute(query)

In [101]:
query = "DROP table allHands"
rows = session.execute(query)

### Close the session and cluster connection¶

In [102]:
session.shutdown()
cluster.shutdown()